In [1]:
from pyspark.sql import SparkSession
from delta import *
from termcolor import cprint

builder = SparkSession.builder.appName("mapPartitions") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/09 15:50:13 WARN Utils: Your hostname, javier-ubuntu, resolves to a loopback address: 127.0.1.1; using 172.17.0.1 instead (on interface docker0)
25/08/09 15:50:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/javier/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/javier/.ivy2.5.2/cache
The jars for the packages stored in: /home/javier/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-77548cdb-2a27-4a84-b249-d70fbc9419c8;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
:: resolution report :: resolve 155ms :: artifacts dl 7ms
	:: modules in use:
	i

In [2]:
def generator(partition):
    """
    Function yielding some result created by some function applied to each row of a partition (in this case lower-casing a string)

    @partition: iterator-object of partition
    """

    for row in partition:
        yield [word.lower() for word in row["text"]]


df = spark.createDataFrame([(["TEST-A"], ), (["TEST-B"], ),(["TEST-C"], ), (["TEST-D"], ),(["TEST-E"], ), (["TEST-F"], )], ["text"])
df.show()
df = df.repartition(3)

# mapPartitions applies a function on each partition of an RDD. Hence, parallelization can be used if the partitions are distributed over different nodes.
df.rdd.mapPartitions(generator).toDF(["text"]).show()


+--------+
|    text|
+--------+
|[TEST-A]|
|[TEST-B]|
|[TEST-C]|
|[TEST-D]|
|[TEST-E]|
|[TEST-F]|
+--------+

+------+
|  text|
+------+
|test-c|
|test-e|
|test-a|
|test-b|
|test-d|
|test-f|
+------+



### mapPartitions(f, preservesPartitioning=False)
Return a new RDD by applying a function to each partition of this RDD.

In [ ]:

rdd = spark.sparkContext.parallelize([5, 2, 6, 4], 2)   #  parelize in 2 partitions [5, 2]  and [6, 4]

def f(iterator): 
    yield sum(iterator)

rdd.mapPartitions(f).collect()


[7, 10]

### mapPartitionsWithIndex(f, preservesPartitioning=False)
Return a new RDD by applying a function to each partition of this RDD, while tracking the index of the original partition.

In [9]:
rdd = spark.sparkContext.parallelize([1, 2, (3, 4), 5], 4)

def f(splitIndex, iterator): 
    yield splitIndex

partition_indexes = rdd.mapPartitionsWithIndex(f)

# Collect and print the results
print(partition_indexes.collect())

[0, 1, 2, 3]
